In [ ]:
# %% -*- coding: utf-8 -*-

""" Created on June 12, 2021 // @author: Sarah Shi """

# Import and load Python packages
import os
import glob
import numpy as np
import pandas as pd
!pip install PyIRoGlass
import PyIRoGlass as pig

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rc, cm

from google.colab import files

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Load sample FTIR spectra and chemistry/thickness files for reference. 
# For downloading the file
if not os.path.exists('COLAB_BINDER.zip'):
    !wget https://github.com/sarahshi/PyIRoGlass/raw/main/Inputs/COLAB_BINDER.zip

# For unzipping the file
if not os.path.exists('COLAB_BINDER'):
    !unzip COLAB_BINDER.zip


We've loaded in sample data, with transmission and reflectance FTIR, from the GitHub repository for data format referencing. Examples workflows working with these spectra can be found on the [ReadTheDocs](https://pyiroglass.readthedocs.io/en/latest/). 

The Google Colab implementation here aims to get your transmission FTIR spectra processed. We remove degrees of freedom to simplify the process. 

The files necessary include all of your transmission FTIR spectra, as well as 'ChemThick.csv' file, containing all chemistry and thickness data. Find an example [here](https://github.com/sarahshi/PyIRoGlass/blob/main/Inputs/COLAB_BINDER/Colab_Binder_ChemThick.csv). Ensure that all of your transmission FTIR spectra names (before the .csv) match up identically with the sample names in the chemistry and thickness file. This is necessary to ensure that the algorithm can match up samples. 

# Load transmission FTIR spectra here: 

In [ ]:
# Specify the directory where you want to save the uploaded files
desired_spectra_directory = "/content/YOUR_DATA/TransmissionSpectra/"

# Create the directory if it doesn't exist
if not os.path.exists(desired_spectra_directory):
    os.makedirs(desired_spectra_directory)

# Change the current working directory
os.chdir(desired_spectra_directory)

# Upload the files
uploadediles = files.upload()


# Load ChemThick.csv here:

In [ ]:
# Specify the directory where you want to save the uploaded files
desired_chemthick_directory = "/content/YOUR_DATA/"

# Change the current working directory
os.chdir(desired_chemthick_directory)

# Upload the files
uploadediles = files.upload()


# Run PyIRoGlass here:

In [ ]:
# Change back to the original directory (e.g., '/content')
os.chdir("/content/YOUR_DATA/")

# Identify all transmission spectra CSVs
FILES = sorted(glob.glob('TransmissionSpectra/' + '*'))

# Load chemistry and thickness file called ChemThick.csv. If you name the file something else, modify it here. 
MICOMP, THICKNESS = pig.Load_ChemistryThickness('ChemThick.csv')

# Load a list of files and a dictionary of transmission FTIR spectra from the files identified earlier. 
DF_FILES, DF_DICT = pig.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)

# Set the output path name desired for your sample. We will call our output 'RESULTS/'. 
OUTPUT = 'RESULTS'
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

# Run PyIRoGlass! 
DF_OUTPUT, FAILURES = pig.Run_All_Spectra(DF_DICT, OUTPUT)

# Save the dataframe outputs from PyIRoGlass. 
DF_OUTPUT.to_csv(OUTPUT + 'DF.csv')

# Indicate your temperature and pressure of analysis. These are the defaults at LDEO. 
T_ROOM = 25 # C
P_ROOM = 1 # Bar

# Run the concentration calculation after determining all peak heights. 
N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = pig.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(OUTPUT + 'H2OCO2.csv')
DENSITY_EPSILON.to_csv(OUTPUT + 'DensityEpsilon.csv')

# Display results here: 

In [ ]:
display(DF_OUTPUT)

In [ ]:
display(MEGA_SPREADSHEET)

In [ ]:
display(DENSITY_EPSILON)